In [20]:
from fastai.tabular.all import *

#Predict whether income exceeds $50K/yr based on census data

path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

##Excel Structure
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (substract the mean and divide by the std)

In [22]:
dls = TabularDataLoaders.from_csv(path/'adult.csv', path=path, y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

splits = RandomSplitter(valid_pct=0.2)(range_of(df))

to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [8]:
dls = to.dataloaders(bs=64)
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,False,30.000000,167476.000346,9.0,<50k
1,Private,HS-grad,Never-married,Exec-managerial,Not-in-family,White,False,27.000000,133425.001928,9.0,<50k
2,Self-emp-not-inc,Bachelors,Married-civ-spouse,Craft-repair,Husband,White,False,33.000000,202729.000379,13.0,>=50k
3,Private,HS-grad,Never-married,Adm-clerical,Other-relative,Black,False,32.000000,175412.999656,9.0,<50k
4,Private,10th,Married-civ-spouse,Exec-managerial,Husband,White,False,62.999999,183607.999973,6.0,<50k
5,Private,Some-college,Married-civ-spouse,Tech-support,Husband,Black,False,43.000000,401623.003670,10.0,>=50k
6,Private,10th,Married-civ-spouse,Other-service,Husband,White,False,53.000000,208302.000684,6.0,<50k
7,Self-emp-inc,Bachelors,Married-civ-spouse,Farming-fishing,Husband,White,False,48.000000,149217.999043,13.0,>=50k
8,Private,Some-college,Never-married,Adm-clerical,Unmarried,White,False,38.000000,188502.999954,10.0,<50k
9,Private,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,False,28.000000,271466.000831,9.0,>=50k


In [9]:
learn = tabular_learner(dls, metrics=accuracy)

learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.367645,0.362389,0.834152,00:07


In [11]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,5.0,10.0,3.0,11.0,1.0,5.0,1.0,0.914549,-1.007323,1.149780,1.0,1.0
1,5.0,16.0,5.0,14.0,2.0,5.0,1.0,-0.111976,0.144377,-0.030633,1.0,0.0
2,5.0,2.0,3.0,15.0,1.0,3.0,1.0,0.254640,0.050851,-1.211046,0.0,0.0
3,5.0,16.0,5.0,5.0,4.0,5.0,1.0,-0.991855,0.010152,-0.030633,0.0,0.0
4,5.0,12.0,3.0,7.0,1.0,5.0,1.0,1.207842,-0.804650,-0.424104,1.0,0.0
5,5.0,12.0,3.0,13.0,1.0,5.0,1.0,-0.478592,-0.571810,-0.424104,1.0,0.0
6,5.0,12.0,1.0,14.0,2.0,5.0,1.0,0.621256,0.583600,-0.424104,0.0,0.0
7,3.0,10.0,3.0,11.0,1.0,5.0,1.0,-0.405269,0.045076,1.149780,1.0,1.0
8,5.0,13.0,3.0,11.0,1.0,5.0,1.0,0.034670,0.379500,1.543251,1.0,1.0


Predict method for one row

In [10]:
row, clas, probs = learn.predict(df.iloc[0])
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101319.999102,12.0,<50k


In [13]:
clas, probs

(tensor(0), tensor([0.5046, 0.4954]))

In [15]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

learn.get_preds(dl=dl)

(tensor([[0.5046, 0.4954],
         [0.4602, 0.5398],
         [0.9805, 0.0195],
         ...,
         [0.5700, 0.4300],
         [0.8111, 0.1889],
         [0.6720, 0.3280]]),
 None)